In [1]:
import nltk
from nltk.corpus import *
import re
from collections import defaultdict, Counter 

__☼ Write a function subsumes() which holds of two feature structures fs1 and fs2 just in case fs1 subsumes fs2.__

In [74]:
def subsumes(fs1, fs2):
    """
    Returns: True if fs1 subsumes fs2
             False otherwise
    """
    
    def add_dict(fs):
        dict_fs = defaultdict()
        for k, v in fs.items():
            if v == nltk.featstruct.FeatDict:
                add_dict(v)
            elif v == str:
                dict_fs[k] == v
        return dict_fs
                    
    fs1_dict = add_dict(fs1)  
    fs2_dict = add_dict(fs2)
    
    if fs1.unify(fs2) == fs2 and (all(elem in list(fs1_dict.keys()) for elem in list(fs2_dict.keys()))):     
        return True
    else:
        return False
    

In [70]:
fs_1 = nltk.FeatStruct("[NUMBER = 74]")

fs_2 = nltk.FeatStruct(NUMBER = 74, STREET = "rue Pascal")

In [76]:
subsumes(fs_2, fs_1)

False

In [77]:
subsumes(fs_1, fs_2)

True

In [88]:
# sanity check with nltk's function subsumes
print(nltk.featstruct.subsumes(fs_1, fs_2))
print(nltk.featstruct.subsumes(fs_2, fs_1))

True
False


__◑ Develop a feature based grammar that will correctly describe the following Spanish noun phrases:__

    un cuadro hermos-o
    un-os cuadro-s hermos-os
    un-a cortina hermos-a
    un-as cortina-s hermos-as

In [3]:
fcfg_spanish = nltk.grammar.FeatureGrammar.fromstring("""
    % start NP
    NP[AGR=?a] -> DET[AGR=?a] N[AGR=?a] Adj[AGR=?a]
    DET[AGR=[NUM=sg, GEN=f]] -> "una" | "Una"
    DET[AGR=[NUM=sg, GEN=m]] -> "un" | "Un"
    DET[AGR=[NUM=pl, GEN=f]] -> "unas" | "Unas"
    DET[AGR=[NUM=pl, GEN=m]] -> "unos" | "Unos"
    N[AGR=[NUM=sg, GEN=f]] -> "casa" | "cortina"
    N[AGR=[NUM=sg, GEN=m]] -> "cuadro" | "chico"
    N[AGR=[NUM=pl, GEN=f]] -> "casas" | "cortinas"
    N[AGR=[NUM=pl, GEN=m]] -> "cuadros" | "chicos"    
    Adj[AGR=[NUM=pl, GEN=f]] -> "hermosas" | "bonitas" | "pequenas"
    Adj[AGR=[NUM=sg, GEN=m]] -> "hermoso" | "bonito" | "pequeno"
    Adj[AGR=[NUM=sg, GEN=f]] -> "hermosa" | "bonita" | "pequena"
    Adj[AGR=[NUM=pl, GEN=m]] -> "hermosos" | "bonitos" | "pequenos"
    """)
    
parser = nltk.parse.FeatureEarleyChartParser(fcfg_spanish)
sent = "Unos cuadros hermosos"
sent2 = "Un chico pequeno"
sent3 = "Una chico bonita"  # incorrect agreement, will not be parsed

In [9]:
for s in [sent, sent2, sent3]:
    for parse in parser.parse(s.split()):
        print(parse, end = "\n\n")

(NP[AGR=[GEN='m', NUM='pl']]
  (DET[AGR=[GEN='m', NUM='pl']] Unos)
  (N[AGR=[GEN='m', NUM='pl']] cuadros)
  (Adj[AGR=[GEN='m', NUM='pl']] hermosos))

(NP[AGR=[GEN='m', NUM='sg']]
  (DET[AGR=[GEN='m', NUM='sg']] Un)
  (N[AGR=[GEN='m', NUM='sg']] chico)
  (Adj[AGR=[GEN='m', NUM='sg']] pequeno))



__◑ Develop your own version of the EarleyChartParser which only prints a trace if the input sequence fails to parse.__

__◑ Consider the feature structures shown in 6.1.__

Work out on paper what the result is of the following unifications. (Hint: you might find it useful to draw the graph structures.)

    fs1 and fs2
    fs1 and fs3
    fs4 and fs5
    fs5 and fs6
    fs5 and fs7
    fs8 and fs9
    fs8 and fs10

In [2]:
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

In [12]:
print("FS1 and FS2\n{}\n\n".format(fs1.unify(fs2)))
print("FS1 and FS3\n{}\n\n".format(fs1.unify(fs3)))
print("FS4 and FS5\n{}\n\n".format(fs4.unify(fs5)))
print("FS5 and FS6\n{}\n\n".format(fs5.unify(fs6)))
print("FS5 and FS7\n{}\n\n".format(fs5.unify(fs7)))
print("FS8 and FS9\n{}\n\n".format(fs8.unify(fs9)))
print("FS8 and FS10\n{}\n\n".format(fs8.unify(fs10)))

FS1 and FS2
[ A = ?x          ]
[                 ]
[ B = [ C = ?x  ] ]
[     [ D = 'd' ] ]


FS1 and FS3
[ A = 'd'         ]
[                 ]
[ B = [ C = 'd' ] ]


FS4 and FS5
[         [ B = 'b'  ] ]
[ A = (1) [ D = ?x   ] ]
[         [ E -> (1) ] ]
[         [ F = ?x   ] ]
[                      ]
[ C -> (1)             ]


FS5 and FS6
[ A = (1) [ D = 'd' ] ]
[                     ]
[ C = [ E -> (1) ]    ]
[     [ F = 'd'  ]    ]


FS5 and FS7
None


FS8 and FS9
[         [ B = 'b' ] ]
[ A = (1) [ D = 'e' ] ]
[         [ G = 'e' ] ]
[                     ]
[ C = [ B = 'e'  ]    ]
[     [ E -> (1) ]    ]


FS8 and FS10
[         [ B = 'b'  ] ]
[ A = (1) [ D = 'b'  ] ]
[         [ E -> (1) ] ]
[         [ G = 'b'  ] ]
[                      ]
[ C -> (1)             ]




__◑ List two feature structures that subsume [A=?x, B=?x].__

In [79]:
f1 = nltk.FeatStruct("[A=?x, B=?x]")
f0 = nltk.FeatStruct("[A = Linguistics]")
f0_2 = nltk.FeatStruct("[B = Physics]")

__◑ Ignoring structure sharing, give an informal algorithm for unifying two feature structures.__

The source code to unify is here https://www.nltk.org/_modules/nltk/featstruct.html#subsumes . I guess that is what they are easking to do...

__◑ Seemingly synonymous verbs have slightly different syntactic properties (Levin, 1993). Consider the patterns of grammaticality for the verbs loaded, filled, and dumped below. Can you write grammar productions to handle such data?__
			
    The farmer loaded the cart with sand		
    The farmer loaded sand into the cart		
    The farmer filled the cart with sand.
    The farmer dumped sand into the cart
    
    *The farmer filled sand into the cart	
    *The farmer dumped the cart with sand		
    

In [14]:
fcfg_verbs = nltk.grammar.FeatureGrammar.fromstring("""
    % start S
    S -> NP VP
    NP -> DET N | N
    VP -> V[ID = ?i] NP PP[ID = ?i]
    PP[ID = ?i] -> P[ID = ?i] NP
    V[ID = t1] -> "filled"
    V[ID = t2] -> "dumped"
    V -> "loaded"
    DET -> "the"
    N -> "farmer" | "cart" | "sand"
    P[ID = t2] -> "into"
    P[ID = t1] -> "with"
   
    """)

In [24]:
parser_vb = nltk.parse.FeatureEarleyChartParser(fcfg_verbs)
sent_1 = "the farmer filled sand into the cart" # ungrammatical
sent_2 = "the farmer dumped the cart with sand" # ungrammatical
sent_3 = "the farmer filled the cart with sand"
sent_4 = "the farmer loaded sand into the cart"

In [25]:
for p in parser_vb.parse(sent_1.split()):  # no parses available
    print(p)

In [26]:
for p in parser_vb.parse(sent_2.split()):  # no parses available
    print(p)

In [27]:
for p in parser_vb.parse(sent_3.split()):
    print(p)

(S[]
  (NP[] (DET[] the) (N[] farmer))
  (VP[]
    (V[ID='t1'] filled)
    (NP[] (DET[] the) (N[] cart))
    (PP[ID='t1'] (P[ID='t1'] with) (NP[] (N[] sand)))))


In [28]:
for p in parser_vb.parse(sent_4.split()):  # no parses available
    print(p)

(S[]
  (NP[] (DET[] the) (N[] farmer))
  (VP[]
    (V[] loaded)
    (NP[] (N[] sand))
    (PP[ID='t2'] (P[ID='t2'] into) (NP[] (DET[] the) (N[] cart)))))


__★ Morphological paradigms are rarely completely regular, in the sense of every cell in the matrix having a different realization. For example, the present tense conjugation of the lexeme walk only has two distinct forms: walks for the 3rd person singular, and walk for all other combinations of person and number. A successful analysis should not require redundantly specifying that 5 out of the 6 possible morphological combinations have the same realization. Propose and implement a method for dealing with this.__

__★ So-called head features are shared between the parent node and head child. For example, TENSE is a head feature that is shared between a VP and its head V child. See (Gazdar, Klein, & and, 1985) for more details. Most of the features we have looked at are head features — exceptions are SUBCAT and SLASH. Since the sharing of head features is predictable, it should not need to be stated explicitly in the grammar productions. Develop an approach that automatically accounts for this regular behavior of head features.__